# <font color=teal>imports</font>
Most processing is performed in python code, and there's a python module to do everything here without manual

In [1]:
import logging

from src import configs
import os
import sys
sys.path.append(os.path.abspath("../src"))



Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
from src.nflverse_transform_job import load_files
from src.pbp_fact import transform_pbp
from src.nflverse_reader_job import URLReader
from src.pbp_participation import transform_pbp_participation
from src.player_stats import transform_player_stats, merge_injuries
from src.player_injuries import prep_player_injuries
from src.player_stats import transform_players
from src.db_utils import load_dims_to_db


# <font color=teal>housekeeping</font>

In [3]:
LOAD_TO_DB = True
database_schema = 'controls'

# Get the logger
logger = configs.configure_logging("pbp_logger")
logger.setLevel(logging.INFO)

# <font color=teal>read data from nflverse<font/>
Step 1.  Read data and store immediately as raw without transformation or change

In [4]:
%%time

reader = URLReader(start_year=2016, last_year=2022, file_type='csv')
urls = reader.download()

2023-07-12 08:59:35,402 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_receiving.csv.gz
2023-07-12 08:59:35,509 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_rushing.csv.gz
2023-07-12 08:59:35,960 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_passing.csv.gz
2023-07-12 08:59:36,010 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/injuries/injuries_2016.csv
2023-07-12 08:59:36,650 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/players/players.csv
2023-07-12 08:59:36,763 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2017_passing.csv.gz
2023-07-12 08:59:37,030 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2017_rushing.csv.gz
2023-07-12 08:59:37,050 - INFO - Success: https

CPU times: user 11.2 s, sys: 2.1 s, total: 13.3 s
Wall time: 17.6 s


---

# <font color=teal>load and transform play by play datasets</font>

### <font color="#9370DB">load</font>

In [5]:

%%time
pbp = load_files(data_subdir='pbp')


2023-07-12 08:59:51,354 - INFO - Reading all files from pbp
2023-07-12 08:59:51,355 - INFO -   + Reading pbp_2022.csv
2023-07-12 08:59:52,522 - INFO -   + Reading pbp_2020.csv
2023-07-12 08:59:53,680 - INFO -   + Reading pbp_2021.csv
2023-07-12 08:59:54,944 - INFO -   + Reading pbp_2019.csv
2023-07-12 08:59:56,133 - INFO -   + Reading pbp_2018.csv
2023-07-12 08:59:57,321 - INFO -   + Reading pbp_2016.csv
2023-07-12 08:59:58,551 - INFO -   + Reading pbp_2017.csv


CPU times: user 7.85 s, sys: 1.54 s, total: 9.39 s
Wall time: 9.92 s


### <font color="#9370DB">transform</font>

In [6]:

%%time
datasets = transform_pbp(pbp)

2023-07-12 09:00:03,885 - INFO - Impute columns to 0
2023-07-12 09:00:04,117 - INFO - impute non binary pbp columns ...
2023-07-12 09:00:05,146 - INFO - Impute columns to 0
2023-07-12 09:00:05,795 - INFO - Impute columns to 0:00
2023-07-12 09:00:07,452 - INFO - Impute columns to NA
2023-07-12 09:00:14,164 - INFO - moving play_id to play_counter, and creating a joinable play_id key
2023-07-12 09:00:15,129 - INFO - Conform key actions like pass, rush, kickoff, etc. and add a single category field called actions... 
2023-07-12 09:00:21,657 - INFO - Validate actions dimension ...
2023-07-12 09:00:21,883 - INFO - Creating new drive dimension...
2023-07-12 09:00:21,962 - INFO - Validate drive_df dimension ...
2023-07-12 09:00:22,221 - INFO - Creating new situations dimension...
2023-07-12 09:00:22,298 - INFO - Validate situation_df dimension ...
2023-07-12 09:00:22,518 - INFO - Creating new metrics dimension...
2023-07-12 09:00:22,582 - INFO - Validate play_metrics_df dimension ...
2023-07-1

CPU times: user 19.9 s, sys: 6.45 s, total: 26.3 s
Wall time: 28 s


---

# <font color=teal>load and transform play by play participation datasets</font>

### <font color="#9370DB">load</font>

In [7]:
%%time
pbp_participation_df = load_files('pbp-participation')


2023-07-12 09:00:29,288 - INFO - Reading all files from pbp-participation
2023-07-12 09:00:29,290 - INFO -   + Reading pbp-participation_2018.csv
2023-07-12 09:00:29,469 - INFO -   + Reading pbp-participation_2019.csv
2023-07-12 09:00:29,645 - INFO -   + Reading pbp-participation_2021.csv
2023-07-12 09:00:29,829 - INFO -   + Reading pbp-participation_2020.csv
2023-07-12 09:00:30,004 - INFO -   + Reading pbp-participation_2022.csv
2023-07-12 09:00:30,178 - INFO -   + Reading pbp-participation_2017.csv
2023-07-12 09:00:30,357 - INFO -   + Reading pbp-participation_2016.csv


CPU times: user 1.2 s, sys: 149 ms, total: 1.35 s
Wall time: 1.36 s


### <font color="#9370DB">transform</font>

In [8]:

player_df, player_events_df = transform_pbp_participation(
    participation_df=pbp_participation_df,
    player_events=datasets['player_events'])

datasets.update({
    'player_participation': player_df,
    'player_events': player_events_df,
})

2023-07-12 09:00:30,649 - INFO - pbp_participation:  move play_id to a play_count column and create a unique play_id that can be used in joins...
2023-07-12 09:00:30,924 - INFO - Calculating defense and offense team names by player and play...
2023-07-12 09:00:33,757 - INFO - Exploding offensive players to their own dataset...
2023-07-12 09:00:35,370 - INFO - Exploding defense_players to their own dataset...
2023-07-12 09:00:45,000 - WARNING - combining offense and defense players - counts are incorrect 6903461 != 6837089


---

# <font color=teal>transform player injuries</font>

### <font color="#9370DB">load</font>

In [9]:
%%time
injuries_df = load_files('injuries')

2023-07-12 09:00:50,001 - INFO - Reading all files from injuries
2023-07-12 09:00:50,003 - INFO -   + Reading injuries_2016.csv
2023-07-12 09:00:50,014 - INFO -   + Reading injuries_2017.csv
2023-07-12 09:00:50,024 - INFO -   + Reading injuries_2020.csv
2023-07-12 09:00:50,036 - INFO -   + Reading injuries_2021.csv
2023-07-12 09:00:50,049 - INFO -   + Reading injuries_2022.csv
2023-07-12 09:00:50,060 - INFO -   + Reading injuries_2019.csv
2023-07-12 09:00:50,071 - INFO -   + Reading injuries_2018.csv


CPU times: user 78 ms, sys: 11.2 ms, total: 89.3 ms
Wall time: 102 ms


### <font color="#9370DB">transform</font>

In [10]:
%%time
injuries_df = prep_player_injuries(injuries_df)

2023-07-12 09:00:50,107 - INFO - Prep injury data...
2023-07-12 09:00:50,107 - INFO - Conforming names (e.g. gsis_id -> player_id)
2023-07-12 09:00:50,118 - INFO - Merge sparse injury columns
2023-07-12 09:00:50,120 - INFO - Get best values for null report_statuses...
2023-07-12 09:00:50,198 - INFO - check that all positions are correct...


CPU times: user 93.9 ms, sys: 4.32 ms, total: 98.2 ms
Wall time: 101 ms


---

# <font color=teal>transform player stats</font>

In [11]:
%%time
stats_df = load_files('player-stats')
stats_df = transform_player_stats(stats_df)
stats_df = merge_injuries(player_stats=stats_df, player_injuries=injuries_df)

2023-07-12 09:00:50,211 - INFO - Reading all files from player-stats
2023-07-12 09:00:50,212 - INFO -   + Reading player-stats.csv
2023-07-12 09:00:50,578 - INFO - fix specific player_stats: <function player_stats_fixes at 0x2a5ba8310>..
2023-07-12 09:00:50,664 - INFO - replace empty position_groups with position info...
2023-07-12 09:00:50,679 - INFO - replace empty player_name with player_display_name info...
2023-07-12 09:00:50,693 - INFO - replace empty headshot_url with 'none'...
2023-07-12 09:00:50,705 - INFO - fillna(0) for all binary columns...
2023-07-12 09:00:50,706 - INFO - Impute columns to 0
2023-07-12 09:00:50,913 - WARNING - After merge player_stats count changed - went from 124081 to 124083


CPU times: user 591 ms, sys: 97 ms, total: 688 ms
Wall time: 707 ms


---

# <font color=teal>direct loads </font>

### <font color="#9370DB">adv stats</font>

In [12]:
%%time

advstats_def_df = load_files('advstats-season-def')
advstats_pass_df = load_files('advstats-season-pass')
advstats_rec_df = load_files('advstats-season-rec')
advstats_rush_df = load_files('advstats-season-rush')


2023-07-12 09:00:50,921 - INFO - Reading all files from advstats-season-def
2023-07-12 09:00:50,922 - INFO -   + Reading advstats-season-def.csv
2023-07-12 09:00:50,931 - INFO - Reading all files from advstats-season-pass
2023-07-12 09:00:50,932 - INFO -   + Reading advstats-season-pass.csv
2023-07-12 09:00:50,935 - INFO - Reading all files from advstats-season-rec
2023-07-12 09:00:50,935 - INFO -   + Reading advstats-season-rec.csv
2023-07-12 09:00:50,941 - INFO - Reading all files from advstats-season-rush
2023-07-12 09:00:50,941 - INFO -   + Reading advstats-season-rush.csv


CPU times: user 21.1 ms, sys: 4.05 ms, total: 25.1 ms
Wall time: 25.5 ms


### <font color="#9370DB">nextgen stats</font>

In [13]:
%%time
next_pass_df = load_files('nextgen-passing')


2023-07-12 09:00:50,949 - INFO - Reading all files from nextgen-passing
2023-07-12 09:00:50,950 - INFO -   + Reading nextgen-passing_2017.csv.gz
2023-07-12 09:00:50,955 - INFO -   + Reading nextgen-passing_2021.csv.gz
2023-07-12 09:00:50,960 - INFO -   + Reading nextgen-passing_2019.csv.gz
2023-07-12 09:00:50,964 - INFO -   + Reading nextgen-passing_2020.csv.gz
2023-07-12 09:00:50,968 - INFO -   + Reading nextgen-passing_2016.csv.gz
2023-07-12 09:00:50,972 - INFO -   + Reading nextgen-passing_2022.csv.gz
2023-07-12 09:00:50,977 - INFO -   + Reading nextgen-passing_2018.csv.gz


CPU times: user 29.2 ms, sys: 3.62 ms, total: 32.8 ms
Wall time: 34.5 ms


In [14]:
%%time
next_rec_df = load_files('nextgen-receiving')


2023-07-12 09:00:50,990 - INFO - Reading all files from nextgen-receiving
2023-07-12 09:00:50,991 - INFO -   + Reading nextgen-receiving_2021.csv.gz
2023-07-12 09:00:50,997 - INFO -   + Reading nextgen-receiving_2017.csv.gz
2023-07-12 09:00:51,003 - INFO -   + Reading nextgen-receiving_2019.csv.gz
2023-07-12 09:00:51,010 - INFO -   + Reading nextgen-receiving_2016.csv.gz
2023-07-12 09:00:51,017 - INFO -   + Reading nextgen-receiving_2020.csv.gz
2023-07-12 09:00:51,023 - INFO -   + Reading nextgen-receiving_2018.csv.gz
2023-07-12 09:00:51,033 - INFO -   + Reading nextgen-receiving_2022.csv.gz


CPU times: user 42.3 ms, sys: 6.2 ms, total: 48.5 ms
Wall time: 52.3 ms


In [15]:
%%time
next_rush_df = load_files('nextgen-rushing')

2023-07-12 09:00:51,045 - INFO - Reading all files from nextgen-rushing
2023-07-12 09:00:51,047 - INFO -   + Reading nextgen-rushing_2018.csv.gz
2023-07-12 09:00:51,051 - INFO -   + Reading nextgen-rushing_2022.csv.gz
2023-07-12 09:00:51,054 - INFO -   + Reading nextgen-rushing_2016.csv.gz
2023-07-12 09:00:51,058 - INFO -   + Reading nextgen-rushing_2020.csv.gz
2023-07-12 09:00:51,061 - INFO -   + Reading nextgen-rushing_2019.csv.gz
2023-07-12 09:00:51,065 - INFO -   + Reading nextgen-rushing_2021.csv.gz
2023-07-12 09:00:51,069 - INFO -   + Reading nextgen-rushing_2017.csv.gz


CPU times: user 25 ms, sys: 3.68 ms, total: 28.7 ms
Wall time: 28.9 ms


### <font color="#9370DB">players</font>

In [16]:
%%time
players_df = load_files('players')
players_df = transform_players(players_df)

2023-07-12 09:00:51,077 - INFO - Reading all files from players
2023-07-12 09:00:51,078 - INFO -   + Reading players.csv
2023-07-12 09:00:51,157 - INFO - Process players dataset...
2023-07-12 09:00:51,158 - INFO - drop players without gsis_ids - they won't link to player_stats
2023-07-12 09:00:51,179 - INFO - fill empty players status to 'NONE'
2023-07-12 09:00:51,186 - INFO - rename gsis_id to player_id...


CPU times: user 93.2 ms, sys: 11.7 ms, total: 105 ms
Wall time: 111 ms


---

# <font color=teal>store to database so we can perform some SQL operations</font>

In [17]:
def load_all_datasets_to_db(data: dict):
    data['schema'] = database_schema
    load_dims_to_db(data)


In [18]:
%%time
if LOAD_TO_DB:
    datasets.update({
        'players': players_df,
        'player_stats': stats_df,
        'adv_stats_def': advstats_def_df,
        'adv_stats_pass': advstats_pass_df,
        'adv_stats_rec': advstats_rec_df,
        'adv_stats_rush': advstats_rush_df,
        'nextgen_pass': next_pass_df,
        'nextgen_rec': next_rec_df,
        'nextgen_rush': next_rush_df
    })
    load_all_datasets_to_db(datasets)

2023-07-12 09:00:51,195 - INFO - create table play_actions in schema controls
2023-07-12 09:01:31,633 - INFO - create table game_drive in schema controls
2023-07-12 09:01:55,961 - INFO - create table play_analytics in schema controls
2023-07-12 09:03:23,119 - INFO - create table play_situations in schema controls
2023-07-12 09:37:27,810 - INFO - create table play_metrics in schema controls
2023-07-12 09:37:55,001 - INFO - create table player_events in schema controls
2023-07-12 09:38:05,301 - INFO - create table game_info in schema controls
2023-07-12 09:38:05,495 - INFO - create table player_participation in schema controls
2023-07-12 09:43:02,222 - INFO - create table players in schema controls
2023-07-12 09:43:03,966 - INFO - create table player_stats in schema controls
2023-07-12 09:43:23,834 - INFO - create table adv_stats_def in schema controls
2023-07-12 09:43:24,259 - INFO - create table adv_stats_pass in schema controls
2023-07-12 09:43:24,319 - INFO - create table adv_stats_r

CPU times: user 4min 16s, sys: 1min 15s, total: 5min 31s
Wall time: 42min 34s
